## `lab13`—Earthquake!

❖ Objectives

-   Use the MATLAB environment to develop and use code.
-   Learn how MATLAB toolboxes add flexibility and functionality to your programming experience.

<div class="alert alert-warning">
**Pair Programming**
<br />
This lab is intended as a *pair* exercise—you need to work in pairs, although you need not be at a single machine unless you prefer to work that way.  At the end, when you report collaborators, please report the names and NetIDs of all partners in this lab exercise.
</div>

In order to ease your transition back-and-forth between Python and MATLAB, we have written a [handout](./lab13-handout.pdf) which covers the basics of translation.  This handout covers many more examples than you have seen in CS 101, but should serve you well throughout your engineering education since both languages are in common currency at the University of Illinois.

### Getting Started

Today you will carry out the lab assignment in MATLAB for the semester. Since the EWS labs have as a default an older version of MATLAB, you should load the newer version as follows (in a new terminal window):

    module switch matlab matlab/R2015a
    matlab

<div class="alert alert-info">
Although you will be using this lab guide in Jupyter, you will write MATLAB code in the MATLAB environment.  Make sure that you modify files in the `lab13/` directory.
</div>

### Seismograms

A seismometer measures the motion of the ground, particularly seismic waves which may originate from earthquakes or volcanic eruptions.  Data often include both the magnitude and the direction of motion, although in our lab today we will only consider magnitude data for simplicity.  The resulting data may be recorded in the form of a *seismogram*, which reflects the simple motion of a needle against a roll of paper.

<table><tr>
<td>
![](https://upload.wikimedia.org/wikipedia/commons/thumb/0/0f/Kinemetrics_seismograph.jpg/800px-Kinemetrics_seismograph.jpg)
</td><td>
![](https://tc1seismometer.files.wordpress.com/2012/07/20120204_200532_bsuz_psn.gif?w=2000&h=)
</td></tr></table>

We will use a generated data set using random numbers for this lab, using a technique you will recreate in the last part.

-   Load the data set from the file `quakedata.txt` using [`importdata`](https://www.mathworks.com/help/matlab/ref/importdata.html), which automatically parses it into an array.  Assign this imported data set the name `seis` (for seismogram).  The first row contains the time values in seconds, and the second row contains the displacement data from the seismometer.
        
        seis = importdata('quakedata.txt');  % load the array
        disp( size(seis) );                  % output the dimensions of the array
    
    After you do this, you will see `seis` appear in the *Workspace* area at right.  `disp` is the MATLAB analogue of `print`, and `size` corresponds to `len`.

If you have loaded everything correctly, this set of tests should pass.  Copy and paste them into the *Command Window* to double-check (you may do this all as a block).

    test_data = importdata('quakedata.txt');  % load the array
    test_size = size(test_data);
    assert(test_size(1) == 2,   'You do not have two rows.')
    assert(test_size(2) == 350, 'You do not have 350 columns.')
    assert(isclose(seis(400), 0.707633859307113), 'You do not appear to have loaded the data.')
    disp('All tests passed successfully.')

Let's see what these data represent as a pictorial seismogram.

-   Plot the second row of `seis` against the first.  Save the plot as `figure_1.png` in your `lab13/` folder.  (You may right-click on the image and `Save image as...`.)
    
    You should use the MATLAB [`plot`](https://www.mathworks.com/help/matlab/ref/plot.html) function.  It works as follows:
        
        plot(x_values, y_values, 'b-');
    
    Since you have to refer to the entire array, you'll have to specify the whole row in the indexing:
        
        plot(seis(1,:), seis(2,:), 'b-');

### Earthquake Detection

An earthquake event can be reconstructed from numerical data by looking at how big the oscillations are—that is, the *variance*.  (Our data set `seis` clearly contains zones where the variance is locally different.)  Mathematically, we prefer to use the square root of the variance, or the *standard deviation*, for convenience.  We define the standard deviation of the elements of a list $X$ as

$$
\text{var}(X) = \sqrt{\text{avg}\left( (X - \bar{X})^2 \right)}
$$

where $\bar{X}$ is the average of the data set.  MATLAB conveniently provides a way to calculate the average value of a list of values:
    
    X = [0.57473 0.09365 0.20067 0.20592]  % define an array
    disp( mean(X) )

MATLAB also supports direct calculation of the standard deviation (and several other statistical quantities):
    
    disp( std(X) )

Suppose that we have a set of seismograph magnitudes and we wish to detect events above a certain threshold.  Using these functions for the average and standard deviation, we can easily calculate the mean and standard deviation for an entire data set, but that doesn't really tell us what we want to know, which is *when* the earthquake threshold event occurred.

![](./img/mean_stdev_all.png)

It is better, in cases like this, to use a *moving-window* or *rolling* function, which instead returns the calculation based on a subset of the entire data set.  For example, a two-point moving average of the data set `[3 4 2 5 1]` would look like `[NaN 3.5 3.0 3.5 3.0]`—the $i$th moving average is the average of the $i-1$th and $i$th values in the original list or array.  More generally, the $n$-point moving average we will define uses the last $n$ data values to calculate an average.  (`np.nan` stands for *n*ot *a* *n*umber, a special data type used when you need the data to be a numeric data type but still not carry an actual value.)  For instance, a twenty-day rolling average of a data set (probably financial, not seismographic) would look like this:

![](./img/mean_rolling.png)

These are very useful for smoothing data sets which contain a lot of noise and variability which obscure the true trends.  You often see them used in financial analyses, for instance, or to filter electronic signals in electrical engineering.

<!--While the average displacement of the earth during a quake is likely still close to zero, the standard deviation of the earth's movement is much larger than baseline data:-->

This data set has a fairly consistent standard deviation (look at the $y$ axis):

![](./img/stdev_rolling.png)

Together, the rolling mean and rolling standard deviation can actually give us a pretty good picture of the statistical behavior of a data set:

![](./img/all_rolling.png)

-   Compose a function `rolling_mean` which accepts as parameters an array called `data` containing a row of time values and a row of measurements (just like `seis`), and a window size `n`.  The function should return the resulting array of time points and calculated rolling-window means.
    
    You will carry this out in the file `rolling_mean.m`, which is located in the `lab13/` directory.  Find it in the left-hand *Current Folder* pane and double-click it to open it.  MATLAB opens a text editor.  Modify the provided code to perform the calculation.  You can test it in the *Command Window* still open below the text editor panel.
    
    We introduce the notion here of a *stencil*—that is, how the values in an array or a grid relate to each other in a particular calculation.  For instance, to calculate the rolling mean with a value of $n = 5$, you require the value at the current index (in red) and two array values behind and two ahead (in yellow):
    
    <img src="./img/stencil.png" width="50%;"/>
    
    Thus at the edges of the array, we will have undefined values since we can't calculate the five-point rolling mean with only three or four points.

If you have loaded everything correctly, this set of tests should pass.  Copy and paste them into the *Command Window* to double-check (you may do this all as a block).
    
    test_data = importdata('quakedata.txt');  % load the array
    test_means = rolling_mean(test_data, 7);  % calculate the rolling means
    test_size = size(test_means);
    assert(test_size(1) == 2,   'You do not have two rows.')
    assert(test_size(2) == 350, 'You do not have 350 columns.')
    assert(isclose(test_means(2,100), 4.994080729574138), 'Your calculation is incorrect.')
    plot(seis(1,:),seis(2,:),'b-',test_means(1,:),test_means(2,:),'r-')
    disp('All tests passed successfully.')

-   Compose a function `rolling_std` which accepts as parameters an array called `data` containing a row of time values and a row of measurements (just like `seis`), and a window size `n`.  The function should return the resulting array of time points and calculated rolling-window standard deviations.
    
    You will carry this out in the file `rolling_std.m`, which is located in the `lab13/` directory.  Open it in a text editor and modify the code to perform the calculation.

If you have loaded everything correctly, this set of tests should pass.  Copy and paste them into the *Command Window* to double-check (you may do this all as a block).
    
    test_data = importdata('quakedata.txt');  % load the array
    test_stds = rolling_std(test_data, 7);    % calculate the rolling means
    test_size = size(test_stds);
    assert(test_size(1) == 2,   'You do not have two rows.')
    assert(test_size(2) == 350, 'You do not have 350 columns.')
    assert(isclose(test_stds(2,100), 7.983935137385763), 'Your calculation is incorrect.')
    plot(seis(1,:),seis(2,:),'b-',test_stds(1,:),test_stds(2,:),'r--')
    disp('All tests passed successfully.')

With `rolling_std` in tow, it should be straightforward to identify threshold events (earthquakes or smaller tremors) against a low-noise background.

Examine the data to determine the first point (index) at which the standard deviation exceeds 5.0.  We will designate points like these as the *beginnings* of events.  (There's nothing magical about the value 5.0; it's just much larger than the standard deviation of the preceding run of values.)

Let's make an attractive plot with earthquake and aftershock data clearly highlighted.  MATLAB can filter the data by value, meaning that we can select highly-variable data from a lower-variability background and plot them separately.
    
    rm = rolling_mean(seis, 5)
    rs = rolling_std(seis, 5)
    
    plot(seis(1,:), seis(2,:),       'b-', ...
         rs(1,:),   rm(2,:),         'r-', ...
         rs(1,:),   rm(2,:)-rs(2,:), 'r--', ...
         rs(1,:),   rm(2,:)+rs(2,:), 'r--')
    
    title('Earthquake Seismogram Data')
    ylabel('Data and Rolling Mean')
    xlabel('time (s)')

-   Save the plot generated by the foregoing code as `figure_2.png` in your `lab13/` folder.

Finally, we can use features of seismogram data to calculate (for instance) how far away the earthquake is.  There is typically a lapse between the primary waves (P), the secondary waves (S) and the surface waves<sup>[[RochaND](http://academic.brooklyn.cuny.edu/geology/grocha/plates/platetec19.htm)]</sup>.  The time lapse between P and S tells us how far away the earthquake occurred (given certain assumptions and observations about the interior of the earth), and the combination of several seismograms from different locations tells us the magnitude and location<sup>[[RochaND](http://academic.brooklyn.cuny.edu/geology/grocha/plates/platetec20.htm)]</sup>.  That process is fairly involved, so you should simply calculate the S–P time interval from the change in rolling variance.

![](./img/earthquake_data.png)

At what point (in your data) does the standard deviation first exceed 5.0?  This is point P.  The next time it falls to a small value for a while and then jumps back up above 5.0 represents point S.  This lapse is the value you should use for the S–P interval.

What is the distance of the earthquake in miles?  To obtain this value, simply multiply the S–P interval in seconds by 5.7.  (This is a gross oversimplification generally, but is adequate for the current problem.<sup>[[TongND](http://www.oakton.edu/user/4/billtong/eas100lab/lab10quake.htm)]</sup>)

-   Compose a function `distance` which accepts as parameters an array called `data` containing a row of time values and a row of measurements (just like `seis`).  The function should return the calculated distance in miles.
    
    You will carry this out in the file `distance.m`, which is located in the `lab13/` directory.  Open it in a text editor and modify the code to perform the calculation.

If you have loaded everything correctly, this set of tests should pass.  Copy and paste them into the *Command Window* to double-check (you may do this all as a block).
    
    test_data = importdata('quakedata.txt');  % load the array
    test_dist = distance(test_data)
    assert(isclose(test_dist, 598.5), 'Your calculation is incorrect.')
    disp('All tests passed successfully.')

###  Making Your Own Earthquake (Data)

Let's compose a function to generate a data set like that above.  While not as exciting as a real earthquake, it's a lot cleaner of a data set to work with.

We will use a random number generator to generate a list of numbers with a specific standard deviation.  Given a number of times and a standard deviation, we want to create a list of random "measurements" which represent the earth's motion during that time.

We'll make the function more general than this though:  it will accept a list of time lengths and a list of corresponding standard deviations.  This way we can make longer time series of measurements like that used as `seis`.

-   Compose a function `quake` which accepts as parameters a list of time lengths `spans` and a list of corresponding standard deviations `stds`.  The function should return an array of random earthquake data as a row of time points and a row of generated measurements.  (Details are included in the file `quake.m`.)
    
    You will carry this out in the file `quake.m`, which is located in the `lab13/` directory.  Open it in a text editor and modify the code to perform the calculation.

If you have loaded everything correctly, this set of tests should pass.  Copy and paste them into the *Command Window* to double-check (you may do this all as a block).

    % Your code should pass these tests.  Do not edit this cell.
    spans = [60 20 40 45 20 10 40 25 10 10 10 60];
    stds  = [ 1 20 10  1 15  5  1 60 20 10  1  1];
    quakedata = quake(spans,stds);

    % the sum of spans should equal the final length
    assert(size(quakedata,2) == sum(spans),      'Your data set is the wrong size.');
    test_max = max(quakedata,[],2);
    %assert(test_max(2) < sum(quakedata(2,:)), 'Something appears to be wrong with your data.');

    plot(quakedata(1,:), quakedata(2,:), 'b-');